In [1]:
# Read from Cosmos DB analytical store into a Spark DataFrame and display 10 rows from the DataFrame
# To select a preferred list of regions in a multi-region Cosmos DB account, add .option("spark.cosmos.preferredRegions", "<Region1>,<Region2>")

df = spark.read\
    .format("cosmos.olap")\
    .option("spark.synapse.linkedService", "asacosmosdb01")\
    .option("spark.cosmos.container", "UserProfileHTAP")\
    .load()

display(df.limit(10))

StatementMeta(SparkPool01, 7, 2, Finished, Available)

SynapseWidget(Synapse.DataFrame, 9ca4cebd-c841-46bf-a475-38a971153fc0)

In [2]:
unwanted_cols = {'_attachments','_etag','_rid','_self','_ts','collectionType','id'}

# Remove unwanted columns from the columns collection
cols = list(set(df.columns) - unwanted_cols)

profiles = df.select(cols)

display(profiles.limit(10))

StatementMeta(SparkPool01, 7, 3, Finished, Available)

SynapseWidget(Synapse.DataFrame, 98d8df73-050c-4b83-a296-7a36dae452ee)

In [3]:
profiles.count()

StatementMeta(SparkPool01, 7, 4, Finished, Available)

99999

In [4]:
from pyspark.sql.functions import udf, explode

preferredProductsFlat=profiles.select('userId',explode('preferredProducts').alias('productId'))
productReviewsFlat=profiles.select('userId',explode('productReviews').alias('productReviews'))
display(productReviewsFlat.limit(10))

StatementMeta(SparkPool01, 7, 5, Finished, Available)

SynapseWidget(Synapse.DataFrame, 51b64e58-e9e0-42a2-b4c9-7633c5e11731)

In [5]:
display(preferredProductsFlat.limit(20))

StatementMeta(SparkPool01, 7, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, 6167835c-91a7-4cc3-a94a-f94d3d19b682)

In [6]:
productReviews = (productReviewsFlat.select('userId','productReviews.productId','productReviews.reviewText')
    .orderBy('userId'))

display(productReviews.limit(10))

StatementMeta(SparkPool01, 7, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, 471b33d3-3efd-4a6e-b09d-b40cd2cd0782)

In [7]:
preferredProductReviews = (preferredProductsFlat.join(productReviews,
    (preferredProductsFlat.userId == productReviews.userId) &
    (preferredProductsFlat.productId == productReviews.productId))
)

display(preferredProductReviews.limit(100))

StatementMeta(SparkPool01, 7, 8, Finished, Available)

SynapseWidget(Synapse.DataFrame, 0e2cfc0a-5deb-4847-8ec6-87469bc8f60c)